In [40]:
import sys,os
sys.path.insert(0, f"{os.path.dirname(os.path.realpath('__file__'))}/../../")
from rxnrecer.config import config as cfg
import pandas as pd
import numpy as np
from types import SimpleNamespace
from Bio import SeqIO
import torch
from rxnrecer.utils import file_utils as futils
from rxnrecer.utils import bio_utils as butils
from rxnrecer.utils import format_utils as format_utils
from rxnrecer.lib.llm import qa as llm_qa
from rxnrecer.cli import predict as predictor
import argparse
import hashlib

import json

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
res_rxnrecer = predictor.step_by_step_prediction(
    input_data=f'{cfg.SAMPLE_DIR}sample10.fasta', 
    output_file=None, 
    output_format='tsv', 
    getEquation=True, 
    mode='s3'
)

Detected input is a FASTA file: /hpcfs/fhome/shizhenkun/codebase/rxnrecer_production/data/sample/sample10.fasta
Step 1: Preparing input data, loading 10 proteins
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 10 proteins


  0%|          | 0/1 [00:00<?, ?it/s]

Running RXNRECer-S1 ...


Predicting reactions: 100%|██████████| 5/5 [00:00<00:00,  7.26it/s]


Running RXNRECer-S3 ...
Transferred model to GPU


Processing Sequences: 100%|██████████| 10/10 [00:00<00:00, 66.98it/s]


Processed 10 proteins in 0.15 seconds
INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 1/1 [02:13<00:00, 133.17s/it]


In [8]:
res_rxnrecer.head(3)

,input_id,RXNRECer-S1,RXNRECer-S2,RXNRECER-S3,RXNRECer_with_prob,RXN_details
0,A0A7H0DNE2,RHEA:14709;RHEA:24076,RHEA:24076;RHEA:14709;RHEA:32187,"[{'reaction_id': 'RHEA:24076', 'selected': 'ye...","{'RHEA:24076': 0.999931, 'RHEA:14709': 0.99993...",[<rxnrecer.lib.rxn.Reaction.Reaction object at...
2,C0HK73,-,-,"[{'reaction_id': '-', 'selected': 'yes', 'rank...",{'-': 0.999998},[]
3,F2XF94,RHEA:32539,RHEA:32539,"[{'reaction_id': 'RHEA:32539', 'selected': 'ye...",{'RHEA:32539': 0.999942},[<rxnrecer.lib.rxn.Reaction.Reaction object at...


In [ ]:
res_rxnrecer.apply(lambda x: format_utils.format_s3_output(x.RXNRECer_with_prob, x['RXNRECER-S3'], x.RXN_details), axis=1)

In [ ]:
sindex = 0
format_utils.format_s3_output(res_rxnrecer.iloc[sindex].RXNRECer_with_prob, res_rxnrecer.iloc[sindex]['RXNRECER-S3'], res_rxnrecer.iloc[sindex].RXN_details)

[{'reaction_id': 'RHEA:24076',
  'prediction_confidence': 0.9999,
  'reaction_details': {'reactants': [{'cpd_id': '',
     'cpd_smiles': 'C12C(C3C(C(CC3)*)(C)CC1)CC=C4C2(CC[C@@H](C4)O)C',
     'cpd_name': 'a 3beta-hydroxy-Delta(5)-steroid',
     'cpd_ref_chebi': 'CHEBI:1722',
     'cpd_link': 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:1722',
     'mol_svg': 'data/cpd_svg/282edb2f820d693364aa28f38cbba53b.svg',
     'cpd_num': 1},
    {'cpd_id': '',
     'cpd_smiles': 'NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O',
     'cpd_name': 'NAD(+)',
     'cpd_ref_chebi': 'CHEBI:57540',
     'cpd_link': 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:57540',
     'mol_svg': 'data/cpd_svg/4764b7413a76a473fe7d46c2f3854173.svg',
     'cpd_num': 1}],
   'products': [{'cpd_id': '',
     'cpd_smiles': 'C12C(C3C(C(CC3)*)(C)CC1)CC=C4C2(CCC(C4)=O)C',
     'cpd_name': 'a 3-oxo-Delta(5)-steroid',
     'cpd_ref_

In [32]:
res_rxnrecer.iloc[sindex].RXNRECer_with_prob['-']

0.999998

In [26]:
res_rxnrecer.iloc[sindex].RXN_details

[]

In [44]:
rxn1 = rxnTool.Reaction(rxn_smiles=res_rxnrecer.reaction_details[0][0]['reaction equation in SMILES format'], 
                        rxn_equation=res_rxnrecer.reaction_details[0][0]['reaction equation'], 
                        rxn_equation_ref_chebi=res_rxnrecer.reaction_details[0][0]['reaction equation in ChEBI format'], 
                        rxn_id=res_rxnrecer.reaction_details[0][0]['reaction id'], 
                        rxn_ec=res_rxnrecer.reaction_details[0][0]['reaction associated Enzyme Commission Number'])


In [17]:
res_rxnrecer.reaction_details[0][0]['reaction equation in ChEBI format']

'CHEBI:47907 = CHEBI:47909'

In [31]:
rxn1.to_dict()

{'reaction_id': 'RHEA:14709',
 'reaction_smiles': 'C12C(C3C(C(CC3)*)(C)CC1)CC=C4C2(CCC(C4)=O)C>>C12C(C3C(C(CC3)*)(C)CC1)CCC=4C2(CCC(C4)=O)C',
 'reaction_equation': 'a 3-oxo-Delta(5)-steroid = a 3-oxo-Delta(4)-steroid',
 'reaction_equation_ref_chebi': 'CHEBI:47907 = CHEBI:47909',
 'reaction_ec': 'EC:5.3.3.1',
 'reactants': [{'cpd_id': '',
   'cpd_smiles': 'C12C(C3C(C(CC3)*)(C)CC1)CC=C4C2(CCC(C4)=O)C',
   'cpd_name': 'a 3-oxo-Delta(5)-steroid',
   'cpd_ref_chebi': 'CHEBI:47907',
   'cpd_link': 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:47907',
   'mol_svg': 'data/cpd_svg/b253cc0ab66e7e78ed6472f9ca0d7c72.svg',
   'cpd_num': 1}],
 'products': [{'cpd_id': '',
   'cpd_smiles': 'C12C(C3C(C(CC3)*)(C)CC1)CCC=4C2(CCC(C4)=O)C',
   'cpd_name': 'a 3-oxo-Delta(4)-steroid',
   'cpd_ref_chebi': 'CHEBI:47909',
   'cpd_link': 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:47909',
   'mol_svg': 'data/cpd_svg/ae78107006d2eda292427f939c8db493.svg',
   'cpd_num': 1}]}